In [5]:
print("1")
import torch
import matplotlib.pyplot as plt
from scipy.special import expit
import sys
from blazeface import FaceExtractor, BlazeFace, VideoReader
from architectures import fornet
from architectures.fornet import FeatureExtractor
from utils import utils
sys.path.append('..')
import torch.nn as nn
from glob import glob

1


In [2]:
net_choices = ['TimmV2','TimmV2ST','ViT','ViTST']

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
face_policy = 'scale'
face_size = 224
frames_per_video = 32

dataset = "ffpp"
net_name = net_choices[0]
net_class = getattr(fornet, net_name)
model_path = "../models/" + dataset + "_"  + "v2.pth"


In [3]:

print("2")
net: FeatureExtractor = net_class().eval().to(device)
net.load_state_dict(torch.load(model_path,map_location='cpu')['net'])
print("3")

transf = utils.get_transformer(face_policy, face_size, net.get_normalizer(), train=False)
facedet = BlazeFace().to(device)
facedet.load_weights("blazeface/blazeface.pth")
facedet.load_anchors("blazeface/anchors.npy")
videoreader = VideoReader(verbose=False)
video_read_fn = lambda x: videoreader.read_frames(x, num_frames=frames_per_video)
face_extractor = FaceExtractor(video_read_fn=video_read_fn,facedet=facedet)
print("4")



2
3
4


In [4]:
video_paths = glob.glob('../sample_videos/ffpp/real/**/*.mp4', recursive = True)
file_names = []
for i in video_paths:
    file_names.append(i.split("/")[4])
file_names.sort()
len(file_names)
file_names

['043.mp4',
 '091.mp4',
 '150.mp4',
 '250.mp4',
 '377.mp4',
 '488.mp4',
 '522.mp4',
 '666.mp4',
 '777.mp4',
 '881.mp4',
 '992.mp4']

In [5]:
video_idxs=[1,3]

input_dir = '../sample_videos/ffpp/real/'


faces = face_extractor.process_videos(input_dir=input_dir , filenames=file_names, video_idxs=video_idxs)
total_videos = len(video_idxs)


faces_frames = [frames_per_video*x for x in range(0,total_videos+1)]   # [0,32,64,96]

faces_hc = torch.stack( [ transf(image=frame['faces'][0])['image'] for frame in faces if len(frame['faces'])] )


predictions = {}
with torch.no_grad():
    for i in range(0,total_videos): #(0,3) i.e 0,1,2
        pred = net(faces_hc[faces_frames[i]:faces_frames[i+1]].to(device)).cpu().numpy().flatten()
        score = expit(pred.mean())
        predictions[input_dir+file_names[video_idxs[i]]] = [round(score,3),'real' if score<0.1 else 'fake']
        predictions[input_dir+file_names[video_idxs[i]]] = [round(score,3),{'predicted_class':'real' if score<0.1 else 'fake','true_class':input_dir.split("/")[3]}]



In [6]:
predictions

{'../sample_videos/ffpp/real/091.mp4': [0.006,
  {'predicted_class': 'real', 'true_class': 'real'}],
 '../sample_videos/ffpp/real/250.mp4': [0.002,
  {'predicted_class': 'real', 'true_class': 'real'}]}

In [11]:
res = list(predictions.values())

In [13]:
predictedClass = res[0][1]['predicted_class']

In [14]:
res

[[0.006, {'predicted_class': 'real', 'true_class': 'real'}],
 [0.002, {'predicted_class': 'real', 'true_class': 'real'}]]

In [16]:
predictedClass

'real'

In [8]:
model_list = []
models_dir = '../models'
model_paths = glob('%s/*.pth'%models_dir)
model_paths

['../models/dfdc_v2.pth',
 '../models/dfdc_v2st.pth',
 '../models/ffpp_vitst.pth',
 '../models/celeb_vit.pth',
 '../models/ffpp_v2.pth',
 '../models/celeb_v2.pth',
 '../models/ffpp_vit.pth',
 '../models/celeb_v2st.pth',
 '../models/ffpp_v2st.pth']

In [9]:
for i in model_paths:
    

AttributeError: 'list' object has no attribute 'split'

In [ ]:
for model_path in model_paths:
    
    net_class = getattr(fornet, net_name)
    
		b3_model = EffNet(arch)
		checkpoint = torch.load(model_path, map_location="cpu")
		b3_model.load_state_dict(checkpoint)
		del checkpoint
		model_list.append(b3_model)

deepware = Ensemble(model_list).eval().to(device)


In [ ]:
class Ensemble(nn.Module):
	def __init__(self, models):
		super(Ensemble, self).__init__()
		self.models = nn.ModuleList(models)

	def forward(self, x):
		preds = []
		for i, model in enumerate(self.models):
			y = model(x)
			preds.append(y)
		final = torch.mean(torch.stack(preds), dim=0)
		return final

In [ ]:
net_choices = ['TimmV2','TimmV2ST','ViT','ViTST']

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
face_policy = 'scale'
face_size = 224
frames_per_video = 32

dataset = "ffpp"
net_name = net_choices[0]
net_class = getattr(fornet, net_name)
model_path = "../models/" + dataset + "_"  + "v2.pth"


print("2")
net: FeatureExtractor = net_class().eval().to(device)
net.load_state_dict(torch.load(model_path,map_location='cpu')['net'])
print("3")
